<a href="https://colab.research.google.com/github/2303A51856/NLP/blob/main/PROJECT_NLP_1856.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers gensim tensorflow

In [ ]:
import pandas as pd

from google.colab import files
uploaded = files.upload()

df = pd.read_csv(list(uploaded.keys())[0])
df.head()

Saving Restaurant reviews.csv to Restaurant reviews.csv


,Restaurant,Reviewer,Review,Rating,Metadata,Time,Pictures,7514
0,Beyond Flavours,Rusha Chakraborty,"The ambience was good, food was quite good . h...",5,"1 Review , 2 Followers",5/25/2019 15:54,0,2447.0
1,Beyond Flavours,Anusha Tirumalaneedi,Ambience is too good for a pleasant evening. S...,5,"3 Reviews , 2 Followers",5/25/2019 14:20,0,NaN
2,Beyond Flavours,Ashok Shekhawat,A must try.. great food great ambience. Thnx f...,5,"2 Reviews , 3 Followers",5/24/2019 22:54,0,NaN
3,Beyond Flavours,Swapnil Sarkar,Soumen das and Arun was a great guy. Only beca...,5,"1 Review , 1 Follower",5/24/2019 22:11,0,NaN
4,Beyond Flavours,Dileep,Food is good.we ordered Kodi drumsticks and ba...,5,"3 Reviews , 2 Followers",5/24/2019 21:37,0,NaN


In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

stop = set(stopwords.words('english'))

def clean_text(t):
    t = re.sub('[^a-zA-Z]', ' ', t)
    t = t.lower()
    t = [w for w in t.split() if w not in stop]
    return " ".join(t)

df['Review'] = df['Review'].astype(str)
df['clean'] = df['Review'].apply(clean_text)
df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Restaurant,Reviewer,Review,Rating,Metadata,Time,Pictures,7514,clean
0,Beyond Flavours,Rusha Chakraborty,"The ambience was good, food was quite good . h...",5,"1 Review , 2 Followers",5/25/2019 15:54,0,2447.0,ambience good food quite good saturday lunch c...
1,Beyond Flavours,Anusha Tirumalaneedi,Ambience is too good for a pleasant evening. S...,5,"3 Reviews , 2 Followers",5/25/2019 14:20,0,NaN,ambience good pleasant evening service prompt ...
2,Beyond Flavours,Ashok Shekhawat,A must try.. great food great ambience. Thnx f...,5,"2 Reviews , 3 Followers",5/24/2019 22:54,0,NaN,must try great food great ambience thnx servic...
3,Beyond Flavours,Swapnil Sarkar,Soumen das and Arun was a great guy. Only beca...,5,"1 Review , 1 Follower",5/24/2019 22:11,0,NaN,soumen das arun great guy behavior sincerety g...
4,Beyond Flavours,Dileep,Food is good.we ordered Kodi drumsticks and ba...,5,"3 Reviews , 2 Followers",5/24/2019 21:37,0,NaN,food good ordered kodi drumsticks basket mutto...


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,classification_report
import pandas as pd

df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')
df.dropna(subset=['Rating'], inplace=True)
df['Rating'] = df['Rating'].round().astype(int)

X = df['clean']
y = df['Rating']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

model_tfidf = SVC(kernel='linear')
model_tfidf.fit(X_train_tfidf,y_train)
pred = model_tfidf.predict(X_test_tfidf)

print("TF-IDF Accuracy:",accuracy_score(y_test,pred))
print(classification_report(y_test,pred))

TF-IDF Accuracy: 0.6302057200200702
              precision    recall  f1-score   support

           1       0.66      0.86      0.75       338
           2       0.33      0.09      0.14       151
           3       0.45      0.32      0.37       228
           4       0.51      0.52      0.51       490
           5       0.74      0.79      0.76       786

    accuracy                           0.63      1993
   macro avg       0.54      0.52      0.51      1993
weighted avg       0.60      0.63      0.61      1993



In [ ]:
from gensim.models import Word2Vec

sentences = [i.split() for i in df['clean']]
w2v = Word2Vec(sentences, vector_size=100, window=5, min_count=1)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['clean'])

X_seq = tokenizer.texts_to_sequences(df['clean'])
maxlen = 100
X_pad = pad_sequences(X_seq, maxlen=maxlen)

y = df['Rating']

X_train,X_test,y_train,y_test = train_test_split(X_pad,y,test_size=0.2,random_state=42)

In [ ]:
import numpy as np

vocab_size = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((vocab_size,100))

for word, idx in tokenizer.word_index.items():
    if word in w2v.wv:
        embedding_matrix[idx] = w2v.wv[word]

In [ ]:
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
import numpy as np
import pandas as pd

y_encoded = pd.get_dummies(y).values

X_train, X_test, y_train_encoded, y_test_encoded = train_test_split(X_pad, y_encoded, test_size=0.2, random_state=42)


model_w2v = Sequential([
    Embedding(vocab_size,100,weights=[embedding_matrix],input_length=maxlen,trainable=False),
    Bidirectional(LSTM(128)),
    Dense(5,activation='softmax')
])

model_w2v.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model_w2v.summary()

model_w2v.fit(X_train,y_train_encoded,epochs=5,batch_size=32,validation_data=(X_test,y_test_encoded))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │     1,448,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,448,300 (5.52 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 1,448,300 (5.52 MB)

Epoch 1/5
249/249 ━━━━━━━━━━━━━━━━━━━━ 73s 280ms/step - accuracy: 0.4775 - loss: 1.2794 - val_accuracy: 0.5113 - val_loss: 1.1461
Epoch 2/5
249/249 ━━━━━━━━━━━━━━━━━━━━ 69s 278ms/step - accuracy: 0.5514 - loss: 1.0933 - val_accuracy: 0.5655 - val_loss: 1.0538
Epoch 3/5
249/249 ━━━━━━━━━━━━━━━━━━━━ 78s 262ms/step - accuracy: 0.5835 - loss: 1.0190 - val_accuracy: 0.5770 - val_loss: 1.0171
Epoch 4/5
249/249 ━━━━━━━━━━━━━━━━━━━━ 65s 262ms/step - accuracy: 0.5852 - loss: 1.0091 - val_accuracy: 0.5800 - val_loss: 1.0167
Epoch 5/5
249/249 ━━━━━━━━━━━━━━━━━━━━ 65s 261ms/step - accuracy: 0.6010 - loss: 0.9606 - val_accuracy: 0.5936 - val_loss: 1.0013


In [ ]:
from transformers import BertTokenizer, TFBertModel

tokenizer_bert = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased', from_pt=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Al

In [ ]:
def bert_encode(texts, tokenizer, max_len=100):
    input_ids, att = [], []

    for t in texts:
        enc = tokenizer.encode_plus(t, max_length=max_len, truncation=True, padding='max_length')
        input_ids.append(enc['input_ids'])
        att.append(enc['attention_mask'])
    return np.array(input_ids), np.array(att)

X_input_ids, X_att = bert_encode(df['clean'], tokenizer_bert)

X_train_ids, X_test_ids, X_train_att, X_test_att, y_train, y_test = train_test_split(
    X_input_ids, X_att, y, test_size=0.2, random_state=42)